## **Loading Libraries**

In [ ]:
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sys
import time

In [ ]:
def returnn(a):
  try:
    return a.string
  except:
    return str(a)

## **Amazon Scraping : Scraping New info from Amazon**

In [ ]:
url_p1 = 'https://www.amazon.in/s?k='
url_p2='&page='
kwords=['facemasks']#,'ppe', 'handsanitizers','handwash']
pages=[1]#,2,3,4,5,6,7,8,9]
amz = pd.DataFrame(columns=['url','product', 'id', 'name', 'mrp', 'discounted'])
for k in kwords:
  for j in pages:
    try:
      quote_page=str(url_p1)+str(k)+str(url_p2)+str(j)
      time.sleep(2)
      page = urlopen(quote_page)
      soup = BeautifulSoup(page, 'html.parser')
      raw_urls = soup.find_all('a', attrs={'class': 'a-link-normal a-text-normal'})
      raw_urls=list(raw_urls)
      for i in raw_urls:
          try:
              time.sleep(2)
              r = requests.get('https://www.amazon.in/' + str(i['href']))
              print(r.url)
              quote_page = str(r.url)
              page = urlopen(quote_page)
              soup = BeautifulSoup(page, 'html.parser')
              pd_name = soup.find('span', attrs={'class': 'a-size-large product-title-word-break'})
              try:
                pd_mrp = soup.find('span', attrs={'class': 'priceBlockStrikePriceString a-text-strike'})
              except:
                pd_mrp='nill'
              try:
                pd_price = soup.find('span', attrs={'class': 'a-size-medium a-color-price priceBlockBuyingPriceString'})
              except:
                pd_price='nill'
              pd_code=r.url
              pd_code=pd_code.split('/')
              pd_code=pd_code[-1].split('?')[0]
              listt=[r.url, k, pd_code, pd_name.string.strip(), returnn(pd_mrp), returnn(pd_price)]
              listt = pd.Series(listt, index = amz.columns)
              amz = amz.append(listt, ignore_index=True)
          except:
              print(sys.exc_info())
    except:
      print(sys.exc_info())
amz


https://www.amazon.in//Marcloire-Reusable-5-Layered-Protection-Multi-Color/dp/B088RNK95K?dchild=1
https://www.amazon.in//Disposable-Packets-Elastic-Ear-Loop-Quality/dp/B073XM2KP3?dchild=1
https://www.amazon.in//GIORDANO-Cotton-Pollution-Reusable-Outdoor/dp/B089FFMMMZ?dchild=1
https://www.amazon.in//Van-Heusen-Mens-Cotton-VHMSBRGPH53287_Assorted_Free/dp/B0883CG5Y9?dchild=1
(<class 'urllib.error.HTTPError'>, <HTTPError 503: 'Service Unavailable'>, <traceback object at 0x7f5a63abaa88>)
https://www.amazon.in//Non-Woven-Disposable-Medical-without-Nose-Pin/dp/B08FJ9JXXW?dchild=1
(<class 'urllib.error.HTTPError'>, <HTTPError 503: 'Service Unavailable'>, <traceback object at 0x7f5a63708d88>)
https://www.amazon.in//Generic-%E0%A4%AF%E0%A5%82%E0%A4%A8%E0%A5%80%E0%A4%B8%E0%A5%87%E0%A4%95%E0%A5%8D%E0%A4%B8-%E0%A4%B0%E0%A5%87%E0%A5%9B%E0%A4%BF%E0%A4%B8%E0%A5%8D%E0%A4%9F%E0%A5%88%E0%A4%82%E0%A4%B8-%E0%A4%B8%E0%A5%8D%E0%A4%AA%E0%A5%8B%E0%A4%B0%E0%A5%8D%E0%A4%9F%E0%A5%8D%E0%A4%B8-%E0%A4%AE%E0%A4%B2%E0

,url,product,id,name,mrp,discounted
0,https://www.amazon.in//Marcloire-Reusable-5-La...,facemasks,B088RNK95K,Marcloire Reusable & Washable 5-Layered Outdoo...,None,₹ 299.00
1,https://www.amazon.in//Disposable-Packets-Elas...,facemasks,B073XM2KP3,SURGICOMFORT Non Woven Elastic Ear-Loop Dispos...,₹ 999.00,₹ 145.80
2,https://www.amazon.in//GIORDANO-Cotton-Polluti...,facemasks,B089FFMMMZ,GIORDANO Cotton Anti Pollution 6 Layer Reusabl...,None,₹ 599.00
3,https://www.amazon.in//Generic-%E0%A4%AF%E0%A5...,facemasks,B07GJR4DLP,Generic Unisex Headwear Headband Head Wrap UV ...,₹ 799.00,₹ 321.00
4,https://www.amazon.in//Vocado-Layer-Filter-Pro...,facemasks,B089K91MK8,Vocado 5 Layer Filter Protection Face Mask Kn9...,₹ 399.00,₹ 121.00
5,https://www.amazon.in//Shocknshop-Multicolor-P...,facemasks,B08D6S5ZFN,Shocknshop Cotton Multicolor Printed Reusable ...,₹ 899.00,₹ 299.00
6,https://www.amazon.in//Gear-Pro-Plus-Face-Blac...,facemasks,B01ALULL9C,"Le Gear Face Mask Pro+ for Bike, Ski, Cycling,...",₹ 700.00,₹ 265.00
7,https://www.amazon.in//Mobistik-Disposable-Lay...,facemasks,B089YNHRP9,Mobistik Disposable Face Mask with 3 Layer Fil...,₹ 399.00,₹ 149.00
8,https://www.amazon.in//YAPZONE-barcelona-print...,facemasks,B08CPHXTD8,YAPZONE FCB barcelona logo printe face mask fo...,₹ 299.00,₹ 139.00


In [ ]:
amz.to_csv (r'amz_scraped_final.csv', index = False, header=True)

## **Scraping present prices using old datasets**

In [ ]:
amz1=pd.read_csv('amazon_2020_03_09_1.csv')
mrp_now=[]
price_now=[]
for i in amz1['asin']:
  try:
    r = requests.get('https://www.amazon.in/dp/' + str(i))
    print(r.url)
    quote_page = str(r.url)
    page = urlopen(quote_page)
    time.sleep(2)
    soup = BeautifulSoup(page, 'html.parser')
    try:
      bbb = soup.find('span', attrs={'class': 'priceBlockStrikePriceString a-text-strike'}) # priceBlockStrikePriceString a-text-strike
      mrp_now.append(bbb.string)
    except:
      mrp_now.append('nill')
    try:
      ccc = soup.find('span', attrs={'class': 'a-size-medium a-color-price priceBlockBuyingPriceString'}) # a-size-medium a-color-price priceBlockBuyingPriceString
      price_now.append(ccc.string)
    except:
      price_now.append('nill')
  except:
    print(sys.exc_info())
    mrp_now.append('nill')
    price_now.append('nill')
amz1['mrp_now']=mrp_now
amz1['price_now']=price_now 

In [ ]:
print(len(amz1['mrp_now']))
amz11=amz1[amz1['mrp_now'] != amz1['mrp_now'][0]]
amz12=amz1[amz1['price_now'] != amz1['mrp_now'][0]]

In [ ]:
amz2=pd.read_csv('amazon_final.csv')
mrp_now=[]
price_now=[]
for i in amz2['asin']:
  try:
    r = requests.get('https://www.amazon.in/dp/' + str(i))
    print(r.url)
    quote_page = str(r.url)
    page = urlopen(quote_page)
    time.sleep(2)
    soup = BeautifulSoup(page, 'html.parser')
    try:
      bbb = soup.find('span', attrs={'class': 'priceBlockStrikePriceString a-text-strike'}) # priceBlockStrikePriceString a-text-strike
      mrp_now.append(bbb.string)
    except:
      mrp_now.append('nill')
    try:
      ccc = soup.find('span', attrs={'class': 'a-size-medium a-color-price priceBlockBuyingPriceString'}) # a-size-medium a-color-price priceBlockBuyingPriceString
      price_now.append(ccc.string)
    except:
      price_now.append('nill')
  except:
    print(sys.exc_info())
    mrp_now.append('nill')
    price_now.append('nill')


amz2['mrp_now']=mrp_now
amz2['price_now']=price_now 

In [ ]:
print(len(amz2['mrp_now']))
amz21=amz2[amz2['mrp_now'] != amz2['mrp_now'][0]]
amz22=amz2[amz2['price_now'] != amz2['mrp_now'][0]]

In [ ]:
frames=[amz11,amz12,amz21,amz22]
result=pd.concat(frames)
result=result.drop_duplicates(subset='asin')
result=result.reset_index(drop=True)
result.to_csv('amazon_pred_final.csv')

In [ ]:
amz30k=pd.read_csv('marketing_sample_for_amazon_in_ecommerce_20191001_20191031_30k_data.csv')
amznew=pd.DataFrame(columns=list(amz30k.columns))
amz30k.reset_index()
words=['hand','soap','sanitizer','wipes', 'wash','handwash','bacteria', 'alcohol', 'hygiene']
for i in words:
    a=[amznew,amz30k[amz30k['product_title'].str.contains(i, regex=False)]]
    amznew=pd.concat(a)
amznew.drop_duplicates(subset ="product_asin", 
                     keep = False, inplace = True)
print(amznew.shape)
amznew.to_csv('amz30k_rel.csv')

In [ ]:
amz3=pd.read_csv('amz30k_rel.csv')
mrp_now=[]
price_now=[]
for i in amz3['product_asin']:
  try:
    r = requests.get('https://www.amazon.in/dp/' + str(i))
    print(r.url)
    quote_page = str(r.url)
    page = urlopen(quote_page)
    soup = BeautifulSoup(page, 'html.parser')
    try:
      bbb = soup.find('span', attrs={'class': 'priceBlockStrikePriceString a-text-strike'}) # priceBlockStrikePriceString a-text-strike
      mrp_now.append(bbb.string)
    except:
      mrp_now.append('nill')
    try:
      ccc = soup.find('span', attrs={'class': 'a-size-medium a-color-price priceBlockBuyingPriceString'}) # a-size-medium a-color-price priceBlockBuyingPriceString
      price_now.append(ccc.string)
    except:
      price_now.append('nill')
  except:
    print(sys.exc_info())
    mrp_now.append('nill')
    price_now.append('nill')


amz3['mrp_now']=mrp_now
amz3['price_now']=price_now 

In [ ]:
amz31=amz3[amz3['mrp_now'] != amz3['mrp_now'][1]]
amz32=amz3[amz3['price_now'] != amz3['mrp_now'][1]]
frames=[amz31,amz32]
result=pd.concat(frames)
result=result.drop_duplicates(subset='product_asin')
result=result.reset_index(drop=True)
result.to_csv('amazon_30k_final.csv')

## **Flipkart Scarping : Scraping New info from Flipkart**

In [ ]:
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
TAG_RE = re.compile(r'<[^>]+>')
kwords=['facemasks','ppe%20kit', 'handsanitizers','handwash']
amz = pd.DataFrame(columns=['url','product', 'name', 'mrp', 'discounted'])
pages=[1,2,3,4,5,6,7,8,9]
def remove_tags(text):
    return TAG_RE.sub('', text)
url_p1 = 'https://www.flipkart.com/search?q='
url_p2='&page='
for j in kwords:
  for k in pages: 
        page = urlopen(str(url_p1)+j+str(url_p2)+str(k))
        soup = BeautifulSoup(page, 'html.parser')
        raw_urls = soup.find_all('a', attrs={'class': '_2cLu-l'})
        raw_urls=list(raw_urls)
        for i in raw_urls:
                try:
                        r = requests.get('https://www.flipkart.com' + str(i['href']))
                        #print(r.url)
                        quote_page = str(r.url)
                        page = urlopen(quote_page)
                        soup = BeautifulSoup(page, 'html.parser')
                        try:
                          aaa = soup.find_all('span', attrs={'class': '_35KyD6'})
                          aaa=remove_tags(str(aaa[0]))
                        except:
                          aaa='nill'
                        try:
                          bbb = soup.find_all('div', attrs={'class': '_3auQ3N _1POkHg'})
                          bbb=remove_tags(str(bbb[0]))
                        except:
                          bbb='nill'
                        try:
                          ccc = soup.find('div', attrs={'class': '_1vC4OE _3qQ9m1'})
                          ccc =ccc.string
                        except:
                          ccc='nill'
                        print(aaa)
                        print(bbb)
                        print(ccc)
                        listt=[r.url, j,aaa,bbb,ccc]
                        listt = pd.Series(listt, index = amz.columns)
                        amz = amz.append(listt, ignore_index=True)
                except:
                        pass
amz






VeBNoR 100 Units Disposable 3 Layer Pharmaceutical Breathable Surgical Pollution Face Mask Respirator with 3 Ply For Men, Women, Kids 3 Layer Pharmaceutical mask (100) Surgical Mask  (Blue, Free Size, Pack of 100, 3 Ply)
₹800
₹249
Wildcraft HypaShield Supermask reusable outdoor protection mask 12535  (Black, L, Pack of 3)
₹600
₹429
Godrej Protekt P-W95 Reusable Face Mask Large Navy 8901023023170  (Blue, L, Pack of 2)
₹480
₹336
NEA COTTON 3-LAYER PROTECTIVE REUSABLE FACE Mask Respirator WASHABLE FABRIC CLOTH MASK Water Resistant, Reusable, Washable  (Orange, Blue, Pink, Free Size, Pack of 3)
₹299
₹249
Van Heusen VHMSBRGB651658 Cloth Mask With Melt Blown Fabric Layer  (Blue, Free Size, Pack of 4)
₹499
₹424
VeBNoR 4 PIECE COMBO OF N95 mask reusable N95 mask washable Respirator Pollution Breathable KN95 mask Face Mask Respirator for Men Women Kids 5 Layers Protection 4 PIECE COMBO OF N95 mask reusable N95 mask washable Respirator Pollution Breathable KN95 mask Face Mask Respirator for Men 

,url,product,name,mrp,discounted
0,https://www.flipkart.com/vebnor-100-units-disp...,facemasks,VeBNoR 100 Units Disposable 3 Layer Pharmaceut...,₹800,₹249
1,https://www.flipkart.com/wildcraft-hypashield-...,facemasks,Wildcraft HypaShield Supermask reusable outdoo...,₹600,₹429
2,https://www.flipkart.com/godrej-protekt-p-w95-...,facemasks,Godrej Protekt P-W95 Reusable Face Mask Large ...,₹480,₹336
3,https://www.flipkart.com/nea-cotton-3-layer-pr...,facemasks,NEA COTTON 3-LAYER PROTECTIVE REUSABLE FACE Ma...,₹299,₹249
4,https://www.flipkart.com/van-heusen-vhmsbrgb65...,facemasks,Van Heusen VHMSBRGB651658 Cloth Mask With Melt...,₹499,₹424
...,...,...,...,...,...
1435,https://www.flipkart.com/mirah-belle-green-app...,handwash,mirah belle Green Apple Hand Wash Can (5 LTR) ...,"₹2,000",₹999
1436,https://www.flipkart.com/intercorp-fomy-antiba...,handwash,Intercorp Fomy Antibacterial Soft Refreshing F...,₹597,₹369
1437,https://www.flipkart.com/himalaya-tulsi-aloe-m...,handwash,Himalaya tulsi and aloe moisturizing hand wash...,₹240,₹178
1438,https://www.flipkart.com/biofresh-fda-approved...,handwash,Biofresh FDA APPROVED Lemon &amp; Tulsi Handwa...,₹600,₹369


In [ ]:
amz.to_csv('flipkart_pandemic_products.csv', index=False)

## **Scraping Amazon Stars**

In [ ]:
amzz=pd.read_csv('amazon_scraped_final.csv')
starlist=[]
count=0
for k in amzz['url'][67:]:
  x=0
  print(str(count)+" ::")
  while (x<=3):
    try:
      time.sleep(2)
      # r = requests.get(k)
      # print(r.url)
      # quote_page = str(r.url)
      page = urlopen(k)
      soup = BeautifulSoup(page, 'html.parser')
      print(k)
      try:
        star1 = soup.find('a', attrs={'class': 'a-link-normal 1star'})
        star1=star1['title']
      except:
        star1='nill'
      try:
        star2 = soup.find('a', attrs={'class': 'a-link-normal 2star'})
        star2=star2['title']
      except:
        star2='nill'
      try:
        star3 = soup.find('a', attrs={'class': 'a-link-normal 3star'})
        star3=star3['title']
      except:
        star3='nill'
      try:
        star4 = soup.find('a', attrs={'class': 'a-link-normal 4star'})
        star4=star4['title']
      except:
        star4='nill'
      try:
        star5 = soup.find('a', attrs={'class': 'a-link-normal 5star'})
        star5=star5['title']
      except:
        star5='nill'
      listt=[star1, star2, star3, star4, star5]
      starlist.append(listt)
      x=4
    except:
      print('try :'+str(x)+' '+str(sys.exc_info()) +' : ' + str(k))
      x=x+1
      if (x == 4):
        starlist.append(['nill', 'nill', 'nill', 'nill', 'nill'])
        print('null appended')
  count=count+1

0 ::
try :0 (<class 'urllib.error.HTTPError'>, <HTTPError 503: 'Service Unavailable'>, <traceback object at 0x7f7ac68bd548>) : https://www.amazon.in//DALUCI-Safety-Protection-Polyester-Material/dp/B08B5VHS2X?dchild=1
try :1 (<class 'urllib.error.HTTPError'>, <HTTPError 503: 'Service Unavailable'>, <traceback object at 0x7f7ac68bd408>) : https://www.amazon.in//DALUCI-Safety-Protection-Polyester-Material/dp/B08B5VHS2X?dchild=1
try :2 (<class 'urllib.error.HTTPError'>, <HTTPError 503: 'Service Unavailable'>, <traceback object at 0x7f7ac68bd308>) : https://www.amazon.in//DALUCI-Safety-Protection-Polyester-Material/dp/B08B5VHS2X?dchild=1
try :3 (<class 'urllib.error.HTTPError'>, <HTTPError 503: 'Service Unavailable'>, <traceback object at 0x7f7ac68bd108>) : https://www.amazon.in//DALUCI-Safety-Protection-Polyester-Material/dp/B08B5VHS2X?dchild=1
null appended
1 ::
try :0 (<class 'urllib.error.HTTPError'>, <HTTPError 503: 'Service Unavailable'>, <traceback object at 0x7f7ac68bd308>) : https:

### **===================================**

In [ ]:
amzz=pd.read_csv('amz_pred_final.csv')
starlist=[]
count=0
for k in amzz['product_url']:
  x=0
  print(str(count)+" ::")
  while (x<=3):
    try:
      time.sleep(2)
      # r = requests.get('https://www.amazon.in/dp/'+k)
      # print(r.url)
      # urll = str(r.url)
      urll=k
      page = urlopen(urll)
      soup = BeautifulSoup(page, 'html.parser')
      print(urll)
      try:
        star1 = soup.find('a', attrs={'class': 'a-link-normal 1star'})
        star1=star1['title']
      except:
        star1='nill'
      try:
        star2 = soup.find('a', attrs={'class': 'a-link-normal 2star'})
        star2=star2['title']
      except:
        star2='nill'
      try:
        star3 = soup.find('a', attrs={'class': 'a-link-normal 3star'})
        star3=star3['title']
      except:
        star3='nill'
      try:
        star4 = soup.find('a', attrs={'class': 'a-link-normal 4star'})
        star4=star4['title']
      except:
        star4='nill'
      try:
        star5 = soup.find('a', attrs={'class': 'a-link-normal 5star'})
        star5=star5['title']
      except:
        star5='nill'
      listt=[star1, star2, star3, star4, star5]
      starlist.append(listt)
      x=4
    except:
      print('try :'+str(x)+' '+str(sys.exc_info()) +' : ' + str(urll))
      x=x+1
      if (x == 4):
        starlist.append(['nill', 'nill', 'nill', 'nill', 'nill'])
        print('null appended')
  count=count+1

0 ::
try :0 (<class 'urllib.error.HTTPError'>, <HTTPError 503: 'Service Unavailable'>, <traceback object at 0x7f075de71ec8>) : https://www.amazon.in/Aurum-Creations-Disposable-Stretchable-100/dp/B0777HKWJF/
try :1 (<class 'urllib.error.HTTPError'>, <HTTPError 503: 'Service Unavailable'>, <traceback object at 0x7f075de8e0c8>) : https://www.amazon.in/Aurum-Creations-Disposable-Stretchable-100/dp/B0777HKWJF/
try :2 (<class 'urllib.error.HTTPError'>, <HTTPError 503: 'Service Unavailable'>, <traceback object at 0x7f075de8e108>) : https://www.amazon.in/Aurum-Creations-Disposable-Stretchable-100/dp/B0777HKWJF/
try :3 (<class 'urllib.error.HTTPError'>, <HTTPError 503: 'Service Unavailable'>, <traceback object at 0x7f075de8e308>) : https://www.amazon.in/Aurum-Creations-Disposable-Stretchable-100/dp/B0777HKWJF/
null appended
1 ::
try :0 (<class 'urllib.error.HTTPError'>, <HTTPError 503: 'Service Unavailable'>, <traceback object at 0x7f075de8e408>) : https://www.amazon.in/Smart-Care-Face-Cloth-Gr

In [ ]:
starlist.count(['nill', 'nill', 'nill', 'nill', 'nill'])

35

In [ ]:
import re
amzz['star1']=[list(map(int, re.findall(r'\d+', i[0])))[0] if (i[0]!='nill') else 0 for i in starlist]
amzz['star2']=[list(map(int, re.findall(r'\d+', i[1])))[0] if (i[1]!='nill') else 0 for i in starlist]
amzz['star3']=[list(map(int, re.findall(r'\d+', i[2])))[0] if (i[2]!='nill') else 0 for i in starlist]
amzz['star4']=[list(map(int, re.findall(r'\d+', i[3])))[0] if (i[3]!='nill') else 0 for i in starlist]
amzz['star5']=[list(map(int, re.findall(r'\d+', i[4])))[0] if (i[4]!='nill') else 0 for i in starlist]

In [ ]:
amzz

,product_name,asin,product_url,brand_name,image_url,mrp_then,price_then,discount_percentage,product_description,date_first_available,number_of_reviews,seller_name,mrp_now,price_now,star1,star2,star3,star4,star5
0,Kodenipr Club Disposable Stretchable Blue Bouf...,B0777HKWJF,https://www.amazon.in/Aurum-Creations-Disposab...,KODENIPR CLUB,https://images-na.ssl-images-amazon.com/images...,299.0,60.0,0.8,Disposable cap is produced using spun bond pol...,2017-08-09T00:00:00,199,Genesis Lifesciences (India) Pvt. Ltd.,249,161,0,0,0,0,0
1,SMARTCARE Face Mask Cloth Tie Green,B079CHH5M3,https://www.amazon.in/Smart-Care-Face-Cloth-Gr...,SMARTCARE,https://images-na.ssl-images-amazon.com/images...,1000.0,250.0,0.75,Cloth Face masks. Used in medical dental lab...,2018-01-28T00:00:00,1,Aastha Uniform and Overgowns,350,314,0,0,0,0,0
2,Salus Disposable Surgical Bouffant Caps 100 Pcs,B0052GIM84,https://www.amazon.in/Salus-Disposable-Surgica...,Salus,https://images-na.ssl-images-amazon.com/images...,249.0,135.0,0.46,100 Pcs Of Disposable Bouffant Cap Qare Made W...,2014-06-04T00:00:00,136,Aurum Creations,250,100,0,0,0,0,0
3,Ionix Pack of 10 Disposable Earloop Medical Fa...,B085LC3VJL,https://www.amazon.in/Ionix-Disposable-Non-Wov...,Ionix,https://images-na.ssl-images-amazon.com/images...,750.0,425.0,0.43,3 Ply face mask Made of premium cotton and n...,2020-03-05T00:00:00,0,Perfect Sales India,599,119,0,0,0,0,0
4,Rudham Disposable Gloves 300 Pieces Transparent,B00LSSNIEI,https://www.amazon.in/Rudham-Disposable-Gloves...,Rudham,https://images-na.ssl-images-amazon.com/images...,199.0,160.0,0.2,300 pieces disposable plastic gloves. For home...,2014-07-14T00:00:00,1484,Cloudtail India,499,153,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,MEDIFIT OT/Surgical Gowns Cap and Mask Large G...,B06Y4LMFB7,https://www.amazon.in/MEDIFIT-Surgical-Gowns-L...,MEDIFIT,https://images-na.ssl-images-amazon.com/images...,830.0,830.0,NaN,This is OT/Surgical Gown which comes along wi...,2017-04-08T00:00:00,6,Medifit Healthcare Garments,nill,₹ 880.00,15,9,0,22,53
67,MEDIFIT Premium Doctor Lab Coat/Apron Six Butt...,B06XB9Y2DR,https://www.amazon.in/MEDIFIT-Premium-Doctor-B...,MEDIFIT,https://images-na.ssl-images-amazon.com/images...,930.0,930.0,NaN,#NAME?,2017-02-20T00:00:00,2,Medifit Healthcare Garments,nill,₹ 980.00,26,28,0,0,46
68,Disposable Air Pollution & Protection Mask Fac...,B085LKYXQ1,https://www.amazon.in/Disposable-Pollution-Pro...,Generic,https://images-na.ssl-images-amazon.com/images...,345.0,270.0,0.22,Cotton face mask which is used to improve the ...,2020-03-06T00:00:00,0,Nihal Cyber Solutions,₹ 40.00,₹ 25.00,0,0,0,0,0
69,Romsons Ventilator Circuit SH-2091 With Fixed ...,B012AQ2XN8,https://www.amazon.in/Romsons-Ventilator-Circu...,Romsons,https://images-na.ssl-images-amazon.com/images...,1100.0,500.0,0.55,Also known as breathing circuit. The purpose o...,2015-07-22T00:00:00,0,shree krishna health care,"₹ 1,100.00",₹ 598.00,0,0,0,0,0


In [ ]:
amzz.to_csv('amazon_pred_final_stars.csv', index=False)

## **Scraping Flipkart Stars**

In [ ]:
import re
import xml.etree.ElementTree


TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return list(xml.etree.ElementTree.fromstring(text).itertext())

In [ ]:
amzz=pd.read_csv('flipkart_pandemic_products.csv')
starlist=[]
count=0
for k in amzz['url']:
  try:
    print(str(count)+" ::")
    #time.sleep(2)
    urll=k
    page = urlopen(urll)
    soup = BeautifulSoup(page, 'html.parser')
    star1 = soup.find('ul', attrs={'class': '_2M5FGu'})
    print(    remove_tags(str(star1)))
    starlist.append(remove_tags(str(star1)))
    print("success : "+str(urll))
  except:
    print("failiure : "+str(urll))
    starlist.append(['nil', 'nil', 'nil', 'nil', 'nil'])
  count=count+1

0 ::
['3,131', '1,638', '1,097', '398', '662']
success : https://www.flipkart.com/vebnor-100-units-disposable-3-layer-pharmaceutical-breathable-surgical-pollution-face-mask-respirator-ply-men-women-kids-100/p/itm7a60af5736e4a?pid=MRPFS2EQGZE3HCHM&lid=LSTMRPFS2EQGZE3HCHM2IXXLP&marketplace=FLIPKART&srno=s_1_1&otracker=search&fm=organic&iid=0b6d1e49-20b8-45f1-91d1-4635b91ffd63.MRPFS2EQGZE3HCHM.SEARCH&ssid=47rvwc0wuo0000001597549158205&qH=50c82b3d02290f82
1 ::
['38,359', '25,056', '11,702', '4,000', '6,134']
success : https://www.flipkart.com/wildcraft-hypashield-supermask-reusable-outdoor-protection-mask-12535/p/itmef53b872f543f?pid=MRPFQN3ZKKEE75ZV&lid=LSTMRPFQN3ZKKEE75ZVMDFUZH&marketplace=FLIPKART&srno=s_1_2&otracker=search&fm=organic&iid=0b6d1e49-20b8-45f1-91d1-4635b91ffd63.MRPFQN3ZKKEE75ZV.SEARCH&ssid=47rvwc0wuo0000001597549158205&qH=50c82b3d02290f82
2 ::
['2,532', '1,570', '666', '211', '258']
success : https://www.flipkart.com/godrej-protekt-p-w95-reusable-face-mask-large-navy-89010

In [ ]:
starlist.count(['nil', 'nil', 'nil', 'nil', 'nil'])

1046

In [ ]:
amzz['star5']=[i[0].replace(',', '') if (i[0]!='nill') else 'nill' for i in starlist]
amzz['star4']=[i[1].replace(',', '') if (i[0]!='nill') else 'nill' for i in starlist]
amzz['star3']=[i[2].replace(',', '') if (i[0]!='nill') else 'nill' for i in starlist]
amzz['star2']=[i[3].replace(',', '') if (i[0]!='nill') else 'nill' for i in starlist]
amzz['star1']=[i[4].replace(',', '') if (i[0]!='nill') else 'nill' for i in starlist]

In [ ]:
amzz.to_csv('flipkart_pandemic_products_with_stars.csv', index=False)